In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/SDA_experiments/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/SDA_experiments/modules']


2024-09-23 04:34:49.662196: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 04:34:49.886749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 04:34:51.091503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets
from datasets import Dataset
import pandas as pd
from datasets import Dataset, DatasetDict


adapter_name="mlm_unipelt_books"

# Define paths to the CSV files
# chose any file ends with category as target
unlabeled_target_data_path = "../../datasets/sa/baby_books/target_unlabelled.csv"
dev_target_data_path = "../../datasets/sa/baby_books/dev_target.csv"
test_target_data_path = "../../datasets/sa/baby_books/test_target.csv"

# Read the CSV files into DataFrames
df_unlabeled_target = pd.read_csv(unlabeled_target_data_path)
df_dev_target = pd.read_csv(dev_target_data_path)
df_test_target = pd.read_csv(test_target_data_path)

# Convert the DataFrames to dictionaries
data_dict_unlabeled_target = df_unlabeled_target.to_dict(orient="records")
data_dict_dev_target = df_dev_target.to_dict(orient="records")
data_dict_test_target = df_test_target.to_dict(orient="records")

# Create Dataset objects from the dictionaries
dataset_unlabeled_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_unlabeled_target],
    
})

dataset_dev_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_dev_target],
})

dataset_test_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_test_target],
    "label": [record["label"] for record in data_dict_test_target]
})

# Combine the unlabeled_target and dev_target datasets into a DatasetDict for training
dataset_train = DatasetDict({
    "train": dataset_unlabeled_target,
    "dev": dataset_dev_target
})

# Print the datasets to verify
print("Training Dataset with Unlabeled and Dev Splits:")
print(dataset_train)

print("\nTest Target Dataset:")
print(dataset_test_target)



Training Dataset with Unlabeled and Dev Splits:
DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 1440
    })
    dev: Dataset({
        features: ['sentence'],
        num_rows: 160
    })
})

Test Target Dataset:
Dataset({
    features: ['sentence', 'label'],
    num_rows: 400
})


In [4]:
reload(config)


train_target = dataset_train
test_target = dataset_test_target
train_target['train'][1]
 

{'sentence': '"i read some raving reviews about this book , but i admit , i was quite disappointed . factual , i \'m sure , but it lacks details and tries to touch on too much without going into any depth . many other pregnancy books go into further detail"'}

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig,UniPELTConfig

union_config = UniPELTConfig()
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_unipelt_books        union            11,083,376      10.178       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 11675504 || all params: 120597674 || trainable%: 9.681367486407739


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 1440
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 160
    })
})

In [8]:
dataset_test_target

Dataset({
    features: ['sentence', 'label'],
    num_rows: 400
})

In [9]:
# we might not need chunks, becasue it is simple sentences
# padding tokens are going to be ignored in the collator 15%, and will be -100 in the lables
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'


In [10]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1440
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 160
    })
})

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(5)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] " [MASK] lord, people must be busier than i [MASK]. [MASK] you have n ‰ t been allowed [MASK] even think about your future after [MASK], then maybe you will get something out [MASK] [MASK] book. for the rest of us, this is drivel " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [13]:

downsampled_dataset = lm_datasets

In [14]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['dev'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/acc

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/5 [00:00<?, ?it/s]

>>> Perplexity: 12.59


In [16]:
trainer.train()

  0%|          | 0/900 [00:00<?, ?it/s]

{'loss': 2.639, 'grad_norm': 1.1707061529159546, 'learning_rate': 9e-06, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.4378108978271484, 'eval_runtime': 1.3106, 'eval_samples_per_second': 122.086, 'eval_steps_per_second': 3.815, 'epoch': 1.0}
{'loss': 2.6191, 'grad_norm': 0.9078348875045776, 'learning_rate': 1.8e-05, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.034545660018921, 'eval_runtime': 2.5793, 'eval_samples_per_second': 62.032, 'eval_steps_per_second': 1.938, 'epoch': 13.0}


Overwriting existing adapter 'mlm_unipelt_books'.
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/adapters/loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any

{'train_runtime': 675.7716, 'train_samples_per_second': 42.618, 'train_steps_per_second': 1.332, 'train_loss': 2.3512400961329796, 'epoch': 13.0}


TrainOutput(global_step=585, training_loss=2.3512400961329796, metrics={'train_runtime': 675.7716, 'train_samples_per_second': 42.618, 'train_steps_per_second': 1.332, 'train_loss': 2.3512400961329796, 'epoch': 13.0})

In [17]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/5 [00:00<?, ?it/s]

>>> Perplexity: 7.84


In [18]:
reload(processed)
testtokenized_dataset= processed.tokenize_dataset(dataset_test_target,tokenizer)
testtokenized_dataset
# we group texts and chunk them
test_dataset = testtokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
test_dataset


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 400
})

In [19]:
import math

# Assuming `test_dataset` is already loaded and processed
# Perform evaluation on the test dataset
test_results = trainer.evaluate(eval_dataset=test_dataset)
# Extract the test loss from the metrics
test_loss = test_results['eval_loss']

# Calculate and print perplexity based on the test loss
print(f">>> Perplexity on the test dataset: {math.exp(test_loss):.2f}")



  0%|          | 0/13 [00:00<?, ?it/s]

>>> Perplexity on the test dataset: 8.04


In [20]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 14.10

The results after:
>>> Perplexity: 4.66